In [1]:
import pandas as pd
import numpy as np
from pronto import Ontology

In [4]:
#Exporting an ontology to the OBO
do = Ontology("../data/HumanDO.obo")

In [7]:
# creeds dataset
creeds = pd.read_csv('../data/Diseases/disease_signatures-v1.0.csv',
                    sep=',', header=0, index_col=0)

In [15]:
creeds.head(20)

cell_type  \
id                                                          
dz:328               Muscle - Striated (Skeletal) (MMHCC)   
dz:325                               Ganglioneuroblastoma   
dz:324                                               Nose   
dz:323                                  Myocardial tissue   
dz:322                  Peripheral blood mononuclear cell   
dz:321                                        Lung Tissue   
dz:320                                       Renal Tissue   
dz:49                                   B Cell Lymphocyte   
dz:822            Back Skin (K5-TGFbeta1 psoriasis model)   
dz:823                Back Skin (K5-Tie2 psoriasis model)   
dz:825             Cumulus cells cultured (Lean patients)   
dz:826            Cumulus cells cultured (Obese patients)   
dz:827  Bronchial epithelial cells - Female - Mild asthma   
dz:445                                        Lung Tissue   
dz:443                                       Hypothalamus   
dz:326                                          Epidermis   
dz:770                                     Neurons - male   
dz:771           Skeletal muscle biopsies - long duration   
dz:305                                  Synovial Membrane   
dz:772          Skeletal muscle biopsies - short duration   

                                                 ctrl_ids      curator  \
id                                                                       
dz:328            GSM4372|GSM4373|GSM4374|GSM4375|GSM4376  Joel.Dudley   
dz:325  GSM282582|GSM282583|GSM282587|GSM282588|GSM282589  Joel.Dudley   
dz:324  GSM286649|GSM286650|GSM286651|GSM286655|GSM286...  Joel.Dudley   
dz:323  GSM82393|GSM82394|GSM82395|GSM82396|GSM82397|G...  Joel.Dudley   
dz:322  GSM66671|GSM66672|GSM66673|GSM66674|GSM66675|G...  Joel.Dudley   
dz:321  GSM533882|GSM533883|GSM533884|GSM533885|GSM533...  Joel.Dudley   
dz:320    GSM9920|GSM9921|GSM9922|GSM9923|GSM9924|GSM9925  Joel.Dudley   
dz:49                 GSM45026|GSM45027|GSM45046|GSM45047  Joel.Dudley   
dz:822                      GSM684707|GSM684708|GSM684709       cadimo   
dz:823                      GSM684679|GSM684680|GSM684681       cadimo   
dz:825  GSM277438|GSM277439|GSM277440|GSM277441|GSM277...       cadimo   
dz:826  GSM277444|GSM277445|GSM277446|GSM277447|GSM277448       cadimo   
dz:827  GSM1068458|GSM1068459|GSM1068460|GSM1068461|GS...       cadimo   
dz:445                         GSM16637|GSM16638|GSM16639  Joel.Dudley   
dz:443       GSM69461|GSM69462|GSM69463|GSM69464|GSM69465  Joel.Dudley   
dz:326            GSM102682|GSM102683|GSM102701|GSM102702  Joel.Dudley   
dz:770            GSM318428|GSM318433|GSM318436|GSM318440       cadimo   
dz:771            GSM303205|GSM303206|GSM303207|GSM303208       cadimo   
dz:305                GSM37430|GSM37431|GSM37432|GSM37433  Joel.Dudley   
dz:772            GSM303205|GSM303206|GSM303207|GSM303208       cadimo   

                             disease_name       do_id    geo_id organism  \
id                                                                         
dz:328        Duchenne muscular dystrophy  DOID:11723    GSE466    mouse   
dz:325                 Nicotine addiction         NaN  GSE11208    human   
dz:324               Rhinovirus infection         NaN  GSE11348    human   
dz:323             dilated cardiomyopathy  DOID:12930   GSE3586    human   
dz:322       bacterial infectious disease    DOID:104   GSE3026    human   
dz:321  idiopathic interstitial pneumonia   DOID:2797  GSE21369    human   
dz:320           type 2 diabetes mellitus   DOID:9352    GSE642    mouse   
dz:49                            leukemia   DOID:1240   GSE2399    human   
dz:822                          psoriasis   DOID:8893  GSE27628    mouse   
dz:823                          psoriasis   DOID:8893  GSE27628    mouse   
dz:825          polycystic ovary syndrome  DOID:11612  GSE10946    human   
dz:826          polycystic ovary syndrome  DOID:11612  GSE10946    human   
dz:827      

In [9]:
#A term, corresponding to a node in the ontology graph.
#create function to get the parent node of a disease
def get_parent_ids(term_name):
    term = do[term_name]
    ancestors = list(term.superclasses())
    ancestor_ids = []
    for a in ancestors:
        ancestor_ids.append(a.id)
    return ancestor_ids

In [10]:
#create function to get the concept ID of a disease
def get_concept_ids(term_name):
    term = do[term_name]
    concept_ids = []
    for xref in list(term.xrefs):
        if 'UMLS_CUI' in str(xref):
            ugly = str(xref).split(':')[1]
            ugly = ugly[:-2]
            concept_ids.append(ugly)
    return concept_ids

In [11]:
# remove null data
DOID =creeds['do_id']
fil = DOID.notna()
DOID = DOID[fil]

In [12]:
total_results = pd.DataFrame()

In [13]:
# match DOID with corresponding concept ID
for id in DOID:
    term_name = do[id]
    term_name = term_name.id
    ancestor_names = get_parent_ids(term_name)
    all_concepts = []
    for ancestor_name_ in ancestor_names:
        all_concepts += get_concept_ids(ancestor_name_)
        results = pd.DataFrame(columns=['doid', 'concept'])
        results['concept'] = all_concepts
        results['doid'] = term_name
    term_name= term_name.replace (':', '-')
    #results.to_csv('../../data/'+term_name+'.csv')
    total_results = pd.concat([total_results, results])
total_results = total_results.reset_index(drop=True)

In [14]:
total_results

doid   concept
0     DOID:11723  C0013264
1     DOID:11723  C0026850
2     DOID:11723  C0026848
3     DOID:11723  C0026857
4     DOID:11723  C0012634
...          ...       ...
4248    DOID:684  C0012243
4249    DOID:684  C0006826
4250    DOID:684  C0012634
4251   DOID:8719  C0007099
4252   DOID:8719  C0012634

[4253 rows x 2 columns]

In [12]:
total_results.to_csv('../data/DOIDs_CONCEPTs.csv')